In [78]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from sklearn.feature_selection import RFE

In [79]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [80]:
##DATA IMPORT

In [81]:
data = {
    'air_visit': pd.read_csv('C:/Users/Monisri/Documents/sem 2/CA675 ass 2/srecruit-restaurant-visitor-forecasting/sair_visit_data.csv'),
    'air_store': pd.read_csv('C:/Users/Monisri/Documents/sem 2/CA675 ass 2/srecruit-restaurant-visitor-forecasting/sair_store_info.csv'),
    'hpg_store': pd.read_csv('C:/Users/Monisri/Documents/sem 2/CA675 ass 2/srecruit-restaurant-visitor-forecasting/shpg_store_info.csv'),
    'air_reserve': pd.read_csv('C:/Users/Monisri/Documents/sem 2/CA675 ass 2/srecruit-restaurant-visitor-forecasting/sair_reserve.csv'),
    'hpg_reserve': pd.read_csv('C:/Users/Monisri/Documents/sem 2/CA675 ass 2/srecruit-restaurant-visitor-forecasting/shpg_reserve.csv'),
    'store_id': pd.read_csv('C:/Users/Monisri/Documents/sem 2/CA675 ass 2/srecruit-restaurant-visitor-forecasting/store_id_relation.csv'),
    'test': pd.read_csv('C:/Users/Monisri/Documents/sem 2/CA675 ass 2/srecruit-restaurant-visitor-forecasting/sample_submission.csv'),
    'holiday': pd.read_csv('C:/Users/Monisri/Documents/sem 2/CA675 ass 2/srecruit-restaurant-visitor-forecasting/sdate_info.csv').rename(columns={'calendar_date':'visit_date'})
    }


In [ ]:
##DATA PREPROCESSING AND FEATURE EXTRACTION

In [82]:
data['hpg_reserve'].head(5)

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [83]:
## Merging hpg_reserve and store_relation_id
data['hpg_reserve'] = pd.merge(data['hpg_reserve'], data['store_id'], how='inner', on=['hpg_store_id'])


In [84]:
for df in ['air_reserve','hpg_reserve']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'reserve_days1', 'reserve_visitors':'reserve_visitors1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'reserve_days2', 'reserve_visitors':'reserve_visitors2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])


In [85]:
data[df].head(5)

,air_store_id,visit_date,reserve_days1,reserve_visitors1,reserve_days2,reserve_visitors2
0,air_00a91d42b08b08d9,2016-01-14,3,2,3.0,2.0
1,air_00a91d42b08b08d9,2016-01-15,6,4,6.0,4.0
2,air_00a91d42b08b08d9,2016-01-16,3,2,3.0,2.0
3,air_00a91d42b08b08d9,2016-01-22,3,2,3.0,2.0
4,air_00a91d42b08b08d9,2016-01-29,6,5,6.0,5.0


In [86]:
data['air_visit']['visit_date'] = pd.to_datetime(data['air_visit']['visit_date'])
data['air_visit']['dow'] = data['air_visit']['visit_date'].dt.dayofweek
data['air_visit']['year'] = data['air_visit']['visit_date'].dt.year
data['air_visit']['month'] = data['air_visit']['visit_date'].dt.month
data['air_visit']['visit_date'] = data['air_visit']['visit_date'].dt.date


In [87]:
data['air_visit'].head(5)

,air_store_id,visit_date,visitors,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1


In [88]:
data['test'].head(5)

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [89]:
data['test']['visit_date'] = data['test']['id'].map(lambda x: str(x).split('_')[2])
data['test']['air_store_id'] = data['test']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['test']['visit_date'] = pd.to_datetime(data['test']['visit_date'])
data['test']['dow'] = data['test']['visit_date'].dt.dayofweek
data['test']['year'] = data['test']['visit_date'].dt.year
data['test']['month'] = data['test']['visit_date'].dt.month
data['test']['visit_date'] = data['test']['visit_date'].dt.date

In [90]:
data['test'].head(5)

,id,visitors,visit_date,air_store_id,dow,year,month
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4


In [91]:
unique_stores = data['test']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

In [92]:
stores.tail(20)

,air_store_id,dow
5727,air_f96765e800907c77,6
5728,air_fa12b40b02fecfd8,6
5729,air_fa4ffc9057812fa2,6
5730,air_fab092c35776a9b1,6
5731,air_fb44f566d4f64a4e,6
5732,air_fbadf737162a5ce3,6
5733,air_fc477473134e9ae5,6
5734,air_fcd4492c83f1c6b9,6
5735,air_fcfbdcf7b1f82c6e,6
5736,air_fd154088b1de6fa7,6


In [93]:
tmp = data['air_visit'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['air_visit'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['air_visit'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['air_visit'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['air_visit'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

In [94]:
stores.head(20)

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0
2,air_0241aa3964b7f861,0,2.0,8.920635,8.0,23.0,63.0
3,air_0328696196e46f18,0,2.0,6.416667,4.0,27.0,12.0
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0
5,air_036d4f1ee7285390,0,4.0,19.600000,19.0,38.0,40.0
6,air_0382c794b73b51ad,0,1.0,20.795455,21.0,47.0,44.0
7,air_03963426c9312048,0,2.0,26.030303,26.0,70.0,66.0
8,air_04341b588bde96cd,0,5.0,35.417910,33.0,76.0,67.0
9,air_049f6d5b402a31b2,0,2.0,9.027778,9.0,20.0,36.0


In [95]:
stores = pd.merge(stores, data['air_store'], how='left', on=['air_store_id']) 

In [96]:
stores.tail(20)

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude
5727,air_f96765e800907c77,6,10.0,15.600000,18.0,20.0,5.0,Izakaya,Hiroshima-ken Hiroshima-shi Kokutaijimachi,34.386245,132.455018
5728,air_fa12b40b02fecfd8,6,11.0,22.243902,21.0,43.0,41.0,Italian/French,Tōkyō-to Meguro-ku Takaban,35.629564,139.684992
5729,air_fa4ffc9057812fa2,6,1.0,6.083333,3.5,27.0,48.0,Dining bar,Tōkyō-to Shinjuku-ku Kabukichō,35.693840,139.703549
5730,air_fab092c35776a9b1,6,4.0,13.264706,11.0,40.0,68.0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Momochi,33.581941,130.348436
5731,air_fb44f566d4f64a4e,6,1.0,17.187500,13.0,45.0,16.0,Izakaya,Tōkyō-to Kōtō-ku Tōyō,35.672853,139.817410
5732,air_fbadf737162a5ce3,6,4.0,25.393939,25.5,51.0,66.0,Izakaya,Tōkyō-to Meguro-ku Kamimeguro,35.641463,139.698171
5733,air_fc477473134e9ae5,6,3.0,17.878049,17.0,59.0,41.0,Bar/Cocktail,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.055460,141.340956
5734,air_fcd4492c83f1c6b9,6,5.0,44.430769,43.0,99.0,65.0,Dining bar,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051
5735,air_fcfbdcf7b1f82c6e,6,1.0,20.400000,20.0,54.0,30.0,Italian/French,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051
5736,air_fd154088b1de6fa7,6,4.0,6.333333,7.0,8.0,3.0,Dining bar,Tōkyō-to Shinjuku-ku Kabukichō,35.693840,139.703549


In [97]:
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = preprocessing.LabelEncoder()
for i in range(4):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name' +str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))

stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

In [98]:
stores.tail(20)

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,air_genre_name0,air_area_name0,air_genre_name1,air_area_name1,air_genre_name2,air_area_name2,air_genre_name3,air_area_name3
5727,air_f96765e800907c77,6,10.0,15.600000,18.0,20.0,5.0,7,15,34.386245,132.455018,7,1,0,5,0,12,0,7
5728,air_fa12b40b02fecfd8,6,11.0,22.243902,21.0,43.0,41.0,6,59,35.629564,139.684992,6,7,2,6,0,25,0,6
5729,air_fa4ffc9057812fa2,6,1.0,6.083333,3.5,27.0,48.0,4,77,35.693840,139.703549,4,7,7,6,0,40,0,6
5730,air_fab092c35776a9b1,6,4.0,13.264706,11.0,40.0,68.0,2,3,33.581941,130.348436,2,0,6,5,0,7,0,7
5731,air_fb44f566d4f64a4e,6,1.0,17.187500,13.0,45.0,16.0,7,55,35.672853,139.817410,7,7,0,6,0,23,0,6
5732,air_fbadf737162a5ce3,6,4.0,25.393939,25.5,51.0,66.0,7,58,35.641463,139.698171,7,7,0,6,0,25,0,6
5733,air_fc477473134e9ae5,6,3.0,17.878049,17.0,59.0,41.0,1,23,43.055460,141.340956,1,2,1,3,0,50,0,4
5734,air_fcd4492c83f1c6b9,6,5.0,44.430769,43.0,99.0,65.0,4,74,35.661777,139.704051,4,7,7,6,0,38,0,6
5735,air_fcfbdcf7b1f82c6e,6,1.0,20.400000,20.0,54.0,30.0,6,74,35.661777,139.704051,6,7,2,6,0,38,0,6
5736,air_fd154088b1de6fa7,6,4.0,6.333333,7.0,8.0,3.0,4,77,35.693840,139.703549,4,7,7,6,0,40,0,6


In [99]:
data['holiday'].head(5)

,visit_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [100]:
data['holiday']['visit_date'] = pd.to_datetime(data['holiday']['visit_date'])
data['holiday']['day_of_week'] = lbl.fit_transform(data['holiday']['day_of_week'])
data['holiday']['visit_date'] = data['holiday']['visit_date'].dt.date


In [101]:
data['holiday'].head(5)

,visit_date,day_of_week,holiday_flg
0,2016-01-01,0,1
1,2016-01-02,2,1
2,2016-01-03,3,1
3,2016-01-04,1,0
4,2016-01-05,5,0


In [102]:
train_data = pd.merge(data['air_visit'], data['holiday'], how='left', on=['visit_date']) 
test_data = pd.merge(data['test'], data['holiday'], how='left', on=['visit_date']) 


In [103]:
train_data = pd.merge(train_data, stores, how='left', on=['air_store_id','dow']) 
test_data = pd.merge(test_data, stores, how='left', on=['air_store_id','dow'])

In [104]:
for df in ['air_reserve','hpg_reserve']:
    train_data = pd.merge(train_data, data[df], how='left', on=['air_store_id','visit_date']) 
    test_data = pd.merge(test_data, data[df], how='left', on=['air_store_id','visit_date'])

In [105]:
train_data['id'] = train_data.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)


In [106]:
train_data['reserve_sum'] = train_data['reserve_visitors1_x'] + train_data['reserve_visitors1_y']
train_data['reserve_mean'] = (train_data['reserve_visitors2_x'] + train_data['reserve_visitors2_y']) / 2
train_data['reserve_date_diff_mean'] = (train_data['reserve_days2_x'] + train_data['reserve_days2_y']) / 2

In [107]:
train_data.head(5)

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,...,reserve_days2_x,reserve_visitors2_x,reserve_days1_y,reserve_visitors1_y,reserve_days2_y,reserve_visitors2_y,id,reserve_sum,reserve_mean,reserve_date_diff_mean
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,6,0,7.0,23.843750,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-13,NaN,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,4,0,2.0,20.292308,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-14,NaN,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,0,0,4.0,34.738462,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-15,NaN,NaN,NaN
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,2,0,6.0,27.651515,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-16,NaN,NaN,NaN
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,1,0,2.0,13.754386,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-18,NaN,NaN,NaN


In [108]:
test_data['reserve_sum'] = test_data['reserve_visitors1_x'] + test_data['reserve_visitors1_y']
test_data['reserve_mean'] = (test_data['reserve_visitors2_x'] + test_data['reserve_visitors2_y']) / 2
test_data['reserve_date_diff_mean'] = (test_data['reserve_days2_x'] + test_data['reserve_days2_y']) / 2


In [109]:
train_data.head(5)

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,...,reserve_days2_x,reserve_visitors2_x,reserve_days1_y,reserve_visitors1_y,reserve_days2_y,reserve_visitors2_y,id,reserve_sum,reserve_mean,reserve_date_diff_mean
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,6,0,7.0,23.843750,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-13,NaN,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,4,0,2.0,20.292308,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-14,NaN,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,0,0,4.0,34.738462,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-15,NaN,NaN,NaN
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,2,0,6.0,27.651515,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-16,NaN,NaN,NaN
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,1,0,2.0,13.754386,...,NaN,NaN,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-18,NaN,NaN,NaN


In [110]:
train_data['date_int'] = train_data['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test_data['date_int'] = test_data['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train_data['var_max_latitude'] = train_data['latitude'].max() - train_data['latitude']
train_data['var_max_longitude'] = train_data['longitude'].max() - train_data['longitude']
test_data['var_max_latitude'] = test_data['latitude'].max() - test_data['latitude']
test_data['var_max_longitude'] = test_data['longitude'].max() - test_data['longitude']

In [111]:
train_data['lon_and_lat'] = train_data['longitude'] + train_data['latitude'] 
test_data['lon_and_lat'] = test_data['longitude'] + test_data['latitude']

lbl = preprocessing.LabelEncoder()
train_data['air_store_id2'] = lbl.fit_transform(train_data['air_store_id'])
test_data['air_store_id2'] = lbl.transform(test_data['air_store_id'])

In [112]:
train_data.head(25)

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,...,reserve_visitors2_y,id,reserve_sum,reserve_mean,reserve_date_diff_mean,date_int,var_max_latitude,var_max_longitude,lon_and_lat,air_store_id2
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,6,0,7.0,23.843750,...,NaN,air_ba937bf13d40fb24_2016-01-13,NaN,NaN,NaN,20160113,8.362564,4.521799,175.409667,603
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,4,0,2.0,20.292308,...,NaN,air_ba937bf13d40fb24_2016-01-14,NaN,NaN,NaN,20160114,8.362564,4.521799,175.409667,603
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,0,0,4.0,34.738462,...,NaN,air_ba937bf13d40fb24_2016-01-15,NaN,NaN,NaN,20160115,8.362564,4.521799,175.409667,603
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,2,0,6.0,27.651515,...,NaN,air_ba937bf13d40fb24_2016-01-16,NaN,NaN,NaN,20160116,8.362564,4.521799,175.409667,603
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,1,0,2.0,13.754386,...,NaN,air_ba937bf13d40fb24_2016-01-18,NaN,NaN,NaN,20160118,8.362564,4.521799,175.409667,603
5,air_ba937bf13d40fb24,2016-01-19,9,1,2016,1,5,0,5.0,18.580645,...,NaN,air_ba937bf13d40fb24_2016-01-19,NaN,NaN,NaN,20160119,8.362564,4.521799,175.409667,603
6,air_ba937bf13d40fb24,2016-01-20,31,2,2016,1,6,0,7.0,23.843750,...,NaN,air_ba937bf13d40fb24_2016-01-20,NaN,NaN,NaN,20160120,8.362564,4.521799,175.409667,603
7,air_ba937bf13d40fb24,2016-01-21,21,3,2016,1,4,0,2.0,20.292308,...,NaN,air_ba937bf13d40fb24_2016-01-21,NaN,NaN,NaN,20160121,8.362564,4.521799,175.409667,603
8,air_ba937bf13d40fb24,2016-01-22,18,4,2016,1,0,0,4.0,34.738462,...,NaN,air_ba937bf13d40fb24_2016-01-22,NaN,NaN,NaN,20160122,8.362564,4.521799,175.409667,603
9,air_ba937bf13d40fb24,2016-01-23,26,5,2016,1,2,0,6.0,27.651515,...,NaN,air_ba937bf13d40fb24_2016-01-23,NaN,NaN,NaN,20160123,8.362564,4.521799,175.409667,603


In [113]:
test_index = test_data['id']
#test_index_idx = test.index
train_data['visitors'] = np.log1p(train_data['visitors'].values)


In [114]:
test_data.head(5)

,id,visitors,visit_date,air_store_id,dow,year,month,day_of_week,holiday_flg,min_visitors,...,reserve_days2_y,reserve_visitors2_y,reserve_sum,reserve_mean,reserve_date_diff_mean,date_int,var_max_latitude,var_max_longitude,lon_and_lat,air_store_id2
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4,3,0,2.0,...,NaN,NaN,NaN,NaN,NaN,20170423,8.326629,4.519803,175.447598,0
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4,1,0,1.0,...,NaN,NaN,NaN,NaN,NaN,20170424,8.326629,4.519803,175.447598,0
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4,5,0,1.0,...,NaN,NaN,NaN,NaN,NaN,20170425,8.326629,4.519803,175.447598,0
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4,6,0,15.0,...,NaN,NaN,NaN,NaN,NaN,20170426,8.326629,4.519803,175.447598,0
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4,4,0,15.0,...,NaN,NaN,NaN,NaN,NaN,20170427,8.326629,4.519803,175.447598,0


In [115]:
#DATA MODELLING

In [116]:
train_data['cat'] = 'train_data'
test_data['cat'] = 'test_data'
    
hot_enc_cols = ['air_genre_name0','air_genre_name1','air_genre_name2','air_genre_name3',
                 'air_area_name0','air_area_name1','air_area_name2','air_area_name3',
                 'air_genre_name','air_area_name','day_of_week','dow','year','month']

total_df = pd.concat((train_data,test_data), axis=0, ignore_index=False)
    
df_index = total_df.index
    
total_df = pd.get_dummies(total_df, columns=hot_enc_cols)

scale_columns = ['lon_and_lat','var_max_longitude','var_max_latitude','date_int','reserve_date_diff_mean','reserve_mean',
             'reserve_sum','reserve_days1_x','reserve_visitors1_x','reserve_days2_x','reserve_visitors2_x','reserve_days1_y','reserve_days2_y','reserve_visitors2_y','latitude','longitude',
             'count_observations','max_visitors','median_visitors','min_visitors','holiday_flg','reserve_visitors1_y',
              'mean_visitors','air_store_id2','date_int','var_max_longitude']

total_df = total_df.fillna(0)


C:\Users\Monisri\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [117]:
from scipy.special import erfinv
def rank_gauss(x):
    N = x.shape[0]
    temp = x.argsort()
    rank_x = temp.argsort() / N
    rank_x -= rank_x.mean()
    rank_x *= 2 
    efi_x = erfinv(rank_x)
    efi_x -= efi_x.mean()
    return efi_x

In [118]:
for coln in scale_columns:
    total_df[coln] = rank_gauss(np.array(total_df[coln]))

total_df.index = df_index    
        
train_data = total_df[total_df['cat']=='train_data']
test_data = total_df[total_df['cat']=='test_data']    
    
drop_columns = ['cat','id', 'air_store_id', 'visit_date','visitors']

In [119]:
final = train_data['visitors']
train_data = train_data.drop(train_data[drop_columns],axis=1)
test_data = test_data.drop(test_data[drop_columns],axis=1)

print('Pre-processing completed')

print('train_data',train_data.shape)
print('test_data',test_data.shape)
print(final.shape)


Pre-processing completed
train_data (252108, 274)
test_data (32019, 274)
(252108,)


In [120]:
from sklearn.model_selection import train_test_split
train_data, valid, y_train, y_valid = train_test_split(train_data, final, test_size=0.15, random_state=2018)

In [121]:
np.random.seed(0)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.optimizers import Adam
import h5py
from keras import backend
from tensorflow import set_random_seed
set_random_seed(99)
    
def rmsle(real, predicted):
    sum=0.0
    for x in range(len(predicted)):
        if predicted[x]<0 or real[x]<0: #check for negative values
            continue
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-05)
    
filepath = 'best_wt_recruit_new.hdf5'

callbacks = [EarlyStopping(monitor='val_loss', patience=30, verbose=0),
             ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=5, verbose=1, epsilon=1e-4, mode='min'),
             ModelCheckpoint(filepath=filepath,monitor='val_loss',save_best_only=True,mode='min')]

   
dropt = .25

C:\Users\Monisri\Anaconda3\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [122]:
model = Sequential()
model.add(Dense(250,activation='relu',input_shape=(train_data.shape[1],)))
model.add(Dropout(dropt)) 
model.add(Dense(30,activation='relu'))
model.add(Dropout(dropt))    
model.add(Dense(1,activation='relu'))

model.compile(loss='mse', optimizer=adam)

In [123]:
feed = model.fit(np.array(train_data), np.array(y_train), epochs=10000, batch_size=200, validation_data=(np.array(valid), np.array(y_valid)),
            verbose=2, callbacks=callbacks, shuffle=False)

model.load_weights('best_wt_recruit_new.hdf5')
    

Train on 214291 samples, validate on 37817 samples
Epoch 1/10000
 - 10s - loss: 0.5463 - val_loss: 0.2749
Epoch 2/10000
 - 7s - loss: 0.4040 - val_loss: 0.2668
Epoch 3/10000
 - 7s - loss: 0.3520 - val_loss: 0.2625
Epoch 4/10000
 - 7s - loss: 0.3118 - val_loss: 0.2605
Epoch 5/10000
 - 7s - loss: 0.2890 - val_loss: 0.2577
Epoch 6/10000
 - 7s - loss: 0.2764 - val_loss: 0.2560
Epoch 7/10000
 - 7s - loss: 0.2695 - val_loss: 0.2551
Epoch 8/10000
 - 7s - loss: 0.2657 - val_loss: 0.2530
Epoch 9/10000
 - 7s - loss: 0.2628 - val_loss: 0.2514
Epoch 10/10000
 - 7s - loss: 0.2604 - val_loss: 0.2500
Epoch 11/10000
 - 7s - loss: 0.2586 - val_loss: 0.2493
Epoch 12/10000
 - 7s - loss: 0.2568 - val_loss: 0.2487
Epoch 13/10000
 - 7s - loss: 0.2559 - val_loss: 0.2487
Epoch 14/10000
 - 7s - loss: 0.2543 - val_loss: 0.2467
Epoch 15/10000
 - 6s - loss: 0.2535 - val_loss: 0.2470
Epoch 16/10000
 - 7s - loss: 0.2522 - val_loss: 0.2471
Epoch 17/10000
 - 7s - loss: 0.2512 - val_loss: 0.2462
Epoch 18/10000
 - 7s -

In [124]:
prediction_value = np.expm1(model.predict(np.array(valid)))
score = rmsle(np.expm1(np.array(y_valid)), prediction_value)

print('score:',score)

score: [0.49171957]


In [125]:
prediction = np.expm1(model.predict(np.array(test_data)))

nn_df = pd.DataFrame(prediction,columns=['visitors'],index=test_index)
print(nn_df.head())
nn_df.to_csv('submit_nn.csv')

print('done')

                                  visitors
id                                        
air_00a91d42b08b08d9_2017-04-23   2.652498
air_00a91d42b08b08d9_2017-04-24  27.266468
air_00a91d42b08b08d9_2017-04-25  34.945644
air_00a91d42b08b08d9_2017-04-26  34.198364
air_00a91d42b08b08d9_2017-04-27  35.408607
done


In [126]:
selector = RFE(feed,10,step=1)
selector = selector.fit(train_data, y_train)
selector.support_ 
selector.ranking_     

TypeError: Cannot clone object '<keras.callbacks.History object at 0x0000022415F56240>' (type <class 'keras.callbacks.History'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.